In [1]:
import pandas as pd

In [2]:
analyst_ratings = pd.read_csv("raw/analyst_ratings_processed.csv")
analyst_ratings.head()

,Unnamed: 0,title,date,stock
0,0.0,Stocks That Hit 52-Week Highs On Friday,2020-06-05 10:30:00-04:00,A
1,1.0,Stocks That Hit 52-Week Highs On Wednesday,2020-06-03 10:45:00-04:00,A
2,2.0,71 Biggest Movers From Friday,2020-05-26 04:30:00-04:00,A
3,3.0,46 Stocks Moving In Friday's Mid-Day Session,2020-05-22 12:45:00-04:00,A
4,4.0,B of A Securities Maintains Neutral on Agilent...,2020-05-22 11:38:00-04:00,A


In [3]:
analyst_ratings["index"] = analyst_ratings["Unnamed: 0"].astype(int)
analyst_ratings.set_index('index')

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

Hay filas con NAs, por lo que hay que revisarlas para eliminarlas o rellenar datos.

In [ ]:
for row, x in enumerate(analyst_ratings["Unnamed: 0"].isna()):
    if x:
        print(analyst_ratings.iloc[row-1:row+1,])

Comprobamos que el problema es que en algunas filas la fecha y stock de la noticia aparece en la siguiente fila en las columnas *title* y *date*. Por lo tanto, lo que hay que hacer es iterar por las filas que tienen NAs en la columna *Unnamed: 0* y meter los valores de las columnas *title* y *date* de esa fila en la anterior.

In [12]:
analyst_ratings_copy = analyst_ratings.copy()
for row, x in enumerate(analyst_ratings["Unnamed: 0"].isna()):
    if x:
        analyst_ratings_copy.iloc[row-1,2] = analyst_ratings.iloc[row,1]  # date
        analyst_ratings_copy.iloc[row-1,3] = analyst_ratings.iloc[row,2]  # stock

Ahora eliminamos las filas con NAs en *Unnamed: 0*, cambiamos el dtype a int, la regeneramos con el número correcto de filas y se la asignamos al index del dataframe.

In [13]:
print(len(analyst_ratings_copy), len([i for i, na in enumerate(analyst_ratings_copy["Unnamed: 0"].isna()) if na]))

analyst_ratings_copy = analyst_ratings_copy.drop([i for i, na in enumerate(analyst_ratings_copy["Unnamed: 0"].isna()) if na])

print(len(analyst_ratings_copy), len([i for i, na in enumerate(analyst_ratings_copy["Unnamed: 0"].isna()) if na]))

analyst_ratings_copy["Unnamed: 0"] = list(range(len(analyst_ratings_copy)))
analyst_ratings_copy.columns = ["index", "title", "date", "stock"]
analyst_ratings_copy = analyst_ratings_copy.set_index("index")

1400469 1289
1399180 0


In [14]:
print(sum(analyst_ratings_copy["title"].isna()))
print(sum(analyst_ratings_copy["date"].isna()))
print(sum(analyst_ratings_copy["stock"].isna()))

0
0
0


De esta forma tenemos el dataframe limpio, sin NAs y listo para fusionarlo con los demás.

In [15]:
analyst_ratings_copy.iloc[:len(analyst_ratings_copy)//2,].to_csv("processed/analyst_ratings_cleaned-1.csv")
analyst_ratings_copy.iloc[len(analyst_ratings_copy)//2:,].to_csv("processed/analyst_ratings_cleaned-2.csv")

In [16]:
pd.read_csv("processed/analyst_ratings_cleaned-1.csv").head()

,index,title,date,stock
0,0,Stocks That Hit 52-Week Highs On Friday,2020-06-05 10:30:00-04:00,A
1,1,Stocks That Hit 52-Week Highs On Wednesday,2020-06-03 10:45:00-04:00,A
2,2,71 Biggest Movers From Friday,2020-05-26 04:30:00-04:00,A
3,3,46 Stocks Moving In Friday's Mid-Day Session,2020-05-22 12:45:00-04:00,A
4,4,B of A Securities Maintains Neutral on Agilent...,2020-05-22 11:38:00-04:00,A


In [17]:
pd.read_csv("processed/analyst_ratings_cleaned-2.csv").head()

,index,title,date,stock
0,699590,Bill Gross: German Bunds Is 'The Short Of A Li...,2015-04-22 15:05:00-04:00,JNS
1,699591,Janus Capital Acquires 3.85% Stake in Isaria W...,2015-04-20 06:39:00-04:00,JNS
2,699592,2 Asset Managers Jefferies Is Watching,2015-04-08 18:41:00-04:00,JNS
3,699593,Bill Gross Reveals 'The Trade of The Last Few ...,2015-04-06 09:44:00-04:00,JNS
4,699594,Bond Fund PIMCO May Lose 'World's Largest' Title,2015-04-03 15:21:00-04:00,JNS
